In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from tpot.builtins import StackingEstimator
from sklearn.feature_selection import VarianceThreshold
from fastai.imports import *
from fastai.structured import *

/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
df_raw = pd.read_feather('tmp/bulldozers-raw')
df_raw.SalePrice = np.log(df_raw.SalePrice)
df, y, nas = proc_df(df_raw, 'SalePrice')

training_features, testing_features, training_target, testing_target = train_test_split(df, y,
                                                    train_size=0.85)

/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
training_target

array([ 9.90349, 10.37349, 10.87805, ..., 10.9682 ,  9.3501 ,  9.4572 ])

In [4]:
exported_pipeline = make_pipeline(
    VarianceThreshold(threshold=0.1),
    RandomForestRegressor(bootstrap=False, max_features=0.25, min_samples_leaf=1, min_samples_split=17, n_estimators=100, n_jobs=8)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


In [8]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

print(rmse(results, testing_target))

0.2018606504118684


In [ ]:
|